# 作業重點:

(1)以, Adam, 為例, 調整 batch_size, epoch , 觀察accurancy, loss 的變化

(2)以同一模型, 分別驗證 SGD, Adam, Rmsprop 的 accurancy

# 作業目標:
    
    取得各種優化器的運算結果

In [12]:
import keras
#from keras.datasets import cifar10
from keras.datasets import mnist 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy

In [13]:
'''
   宣告並設定
   batch_size：對總的樣本數進行分組，每組包含的樣本數量
   epochs ：訓練次數
   
''' 

batch_size =256
epochs=80

In [14]:
#    第一步：選擇模型, 順序模型是多個網絡層的線性堆疊
 
model = Sequential()

#   第二步：構建網絡層
 
model.add(Dense( 500,input_shape=(784,))) # 輸入層，28*28=784   
model.add(Activation('relu')) # 激活函數是relu   

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 500)) # 隱藏層節點500個   
model.add(Activation('relu'))  

model.add(Dense( 10)) # 輸出結果是10個類別，所以維度是10   
model.add(Activation('softmax')) # 最後一層用softmax作為激活函數

In [15]:

# 模型建立完成後，統計參數總量
print("Total Parameters：%d" % model.count_params())

Total Parameters：1149010


In [16]:
'''
 SGD(隨機梯度下降) - Arguments
lr: float >= 0. Learning rate.
momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
decay: float >= 0. Learning rate decay over each update.
nesterov: boolean. Whether to apply Nesterov momentum.
'''

'''
RMSprop- Arguments
lr: float >= 0. Learning rate.
rho: float >= 0.
epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
decay: float >= 0. Learning rate decay over each update.
'''


'\nRMSprop- Arguments\nlr: float >= 0. Learning rate.\nrho: float >= 0.\nepsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().\ndecay: float >= 0. Learning rate decay over each update.\n'

In [17]:

# 第三步：編譯, 
model.compile(optimizer = 'Rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
#  第四步：資料分割
# 使用Keras自帶的mnist工具讀取數據（第一次需要聯網）
(X_train, y_train), (X_test, y_test) = mnist.load_data() 

# 由於mist的輸入數據維度是(num, 28 , 28)，這裡需要把後面的維度直接拼起來變成784維   
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2 ])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2 ])  
Y_train = (numpy.arange(10) == y_train[:, None]).astype(int)
Y_test = (numpy.arange(10) == y_test[:, None]).astype(int)

In [19]:
# 第五步：訓練, 修正 model 參數
#Blas GEMM launch failed , 避免動態分配GPU / CPU, 出現問題
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


history = model.fit(X_train,Y_train,batch_size = batch_size, epochs=epochs, shuffle=True,verbose=2,validation_split=0.3 )
%time

Train on 42000 samples, validate on 18000 samples
Epoch 1/80
 - 2s - loss: 2.8360 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 2/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 3/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 4/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 5/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 6/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 7/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 8/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 9/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 10/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 11/80
 - 1s - loss: 2.8397 - acc: 0.8229 - val_loss: 2.8600 - val_acc: 0.8216
Epoch 12/80
 - 1s - loss: 2.8397 - 

In [20]:
#    第六步：輸出
 
print ( " test set " )
scores = model.evaluate(X_test,Y_test,batch_size=200,verbose= 0)
print ( "" )
#print ( " The test loss is %f " % scores)
print ( " The test loss is %f ", scores)
result = model.predict(X_test,batch_size=200,verbose= 0)

result_max = numpy.argmax(result, axis = 1 )
test_max = numpy.argmax(Y_test, axis = 1 )

result_bool = numpy.equal(result_max, test_max)
true_num = numpy.sum(result_bool)
print ( "" )
print ( " The accuracy of the model is %f " % (true_num/len(result_bool)))

 test set 

 The test loss is %f  [2.8421611833572387, 0.8227000069618225]

 The accuracy of the model is 0.113500 


In [21]:
#test set 
'''
Adam
--------------------
batch_size =128
epochs=20
 The test loss is %f  [2.920068817138672, 0.8178396594524383]
 The accuracy of the model is 0.089200
 
batch_size =256
epochs=20 
The test loss is %f  [2.8764664793014525, 0.8205596566200256]
 The accuracy of the model is 0.102800 
    
batch_size =512
epochs=20 
 The test loss is %f  [2.8764664793014525, 0.8205596566200256]
 The accuracy of the model is 0.102800     
 
batch_size =768 ●●●●
epochs=20 
 The test loss is %f  [0.0331572373189465, 0.992849988937378]
 The accuracy of the model is 0.963900 

batch_size =1024
epochs=20 
 The test loss is %f  [2.8764664793014525, 0.8205596566200256]
 The accuracy of the model is 0.102800 
 
batch_size =128
epochs=20 
 The test loss is %f  [2.8918555402755737, 0.8195996522903443]
 The accuracy of the model is 0.098000 

batch_size =128
epochs=40 
 The test loss is %f  [2.8822373819351195, 0.8201996469497681]
 The accuracy of the model is 0.101000 
batch_size =128
epochs=80
 The test loss is %f  [2.8751840496063235, 0.8206396567821502]
 The accuracy of the model is 0.103200 

batch_size =768
epochs=40
 The test loss is %f  [2.89890887260437, 0.8191596519947052]
 The accuracy of the model is 0.095800 


SGD
--------------------
batch_size =128
epochs=20
 The test loss is %f  [2.8937791919708253, 0.8194796466827392]
 The accuracy of the model is 0.097400 

batch_size =256
epochs=20 
 The test loss is %f  [0.05392159005627036, 0.9888499808311463]
 The accuracy of the model is 0.943400 

batch_size =512
epochs=20 
 The test loss is %f  [2.8751840496063235, 0.8206396567821502]
 The accuracy of the model is 0.103200 

batch_size =768
epochs=20 
 The test loss is %f  [0.07239944168366491, 0.9845299708843231]
 The accuracy of the model is 0.921300 

batch_size =1024
epochs=20 
 The test loss is %f  [2.8912143421173098, 0.8196396505832673]
 The accuracy of the model is 0.098200 
 
batch_size =128
epochs=20 
 The test loss is %f  [2.89890887260437, 0.8191596519947052]
 The accuracy of the model is 0.095800 

batch_size =128
epochs=40 
 The test loss is %f  [2.8751840496063235, 0.8206396567821502]
 The accuracy of the model is 0.103200

batch_size =128 ●●●●
epochs=80
 The test loss is %f  [0.04457024552188159, 0.9905199909210205]
 The accuracy of the model is 0.952400 

batch_size =768
epochs=80


Rmsprop
--------------------
batch_size =128
epochs=20
 The test loss is %f  [2.8764664793014525, 0.8205596566200256]
 The accuracy of the model is 0.102800 
 
batch_size =256
epochs=20  ●●●●
 The test loss is %f  [2.842161717414856, 0.8226996505260468]
 The accuracy of the model is 0.113500 

batch_size =512
epochs=20 
 The test loss is %f  [2.89890887260437, 0.8191596519947052]
 The accuracy of the model is 0.095800 

batch_size =768
epochs=20 
 The test loss is %f  [2.920068817138672, 0.8178396594524383]
 The accuracy of the model is 0.089200

batch_size =128
epochs=40 
 The test loss is %f  [2.920068817138672, 0.8178396594524383]
 The accuracy of the model is 0.089200 

batch_size =128 
epochs=60
 The test loss is %f  [2.882557978630066, 0.8201796531677246]
 The accuracy of the model is 0.100900 

batch_size =256
epochs=80
 The test loss is %f  [2.920068817138672, 0.8178396594524383]
 The accuracy of the model is 0.089200 
'''

'\nAdam\n--------------------\nbatch_size =128\nepochs=20\n The test loss is %f  [2.920068817138672, 0.8178396594524383]\n The accuracy of the model is 0.089200\n \nbatch_size =256\nepochs=20 \nThe test loss is %f  [2.8764664793014525, 0.8205596566200256]\n The accuracy of the model is 0.102800 \n    \nbatch_size =512\nepochs=20 \n The test loss is %f  [2.8764664793014525, 0.8205596566200256]\n The accuracy of the model is 0.102800     \n \nbatch_size =768 ●●●●\nepochs=20 \n The test loss is %f  [0.0331572373189465, 0.992849988937378]\n The accuracy of the model is 0.963900 \n\nbatch_size =1024\nepochs=20 \n The test loss is %f  [2.8764664793014525, 0.8205596566200256]\n The accuracy of the model is 0.102800 \n \nbatch_size =128\nepochs=20 \n The test loss is %f  [2.8918555402755737, 0.8195996522903443]\n The accuracy of the model is 0.098000 \n\nbatch_size =128\nepochs=40 \n The test loss is %f  [2.8822373819351195, 0.8201996469497681]\n The accuracy of the model is 0.101000 \nbatch_s